In [1]:
import torch
from torch.nn.utils.rnn import pack_padded_sequence
import torch.nn as nn
import sentencepiece as spm


In [10]:
#gets all predictions from one window
def get_window_predictions(window, model):
    print(window)
    preds, h = model(torch.tensor([window]))
    preds = torch.flatten(torch.sigmoid(preds))
    preds = preds.detach().numpy()
    return preds

#get_window_predictions(test_x[0],LSTM_LM_net_trained)



#NOTE, THIS ONLY GETS THE PREDICTED BREAK POINTS FROM A PREDICTION
#WITH THE NEWLINE TOKEN AT CENTER OF WINDOW
def get_predicted_break_points(code_windows, model):
    start = 0
    code  =[]
    break_points = []
    mid_points_preds = []
    print(len(code_windows))
    for window_i in range(len(code_windows)):
        #get window, which has our tokens
        window = code_windows[window_i]
        window_predictions = get_window_predictions(window,model)
        #mid = math.ceil(len(window)/2)
        mid = int(len(window)/2) #actually we need to round down...
        mid_token = tokenizer.decode(int(window[mid]))
        mid_pred = window_predictions[mid]
        mid_points_preds.append(mid_pred)
        
        #if only new lines
        #if mid_token[-7:]=='NEWLINE':
        #    if round(mid_pred) == 1:
        #        break_points.append(window_i)
        #print(mid_token,mid_pred)
        if round(mid_pred) == 1:
            print(mid_token, mid_pred)
            break_points.append(window_i)
                
                
        code.append(mid_token)
        start+=1
    
    #print(max(mid_points_preds))
    return code, break_points
    
#code_windows = segments['0']['x']
#code, breaks = get_predicted_break_points(code_windows,LSTM_LM_net_trained)
#print(breaks)

#from code segmentation file
def insert_comments(code, break_spots, comment='\n'+'*'*8+'\n',at_begining=True):
    #if there is a a comment at begining of snippet
    if at_begining:
        #adds a notation to add a 0
        #at beigning of break spots too
        break_spots.insert(0,0)
    
    #go through breaks backwards
    #so as not to mess up break 
    #spots as we would if we went forward
    for b in break_spots[::-1]:
        code.insert(b,comment)
    return code

#get all python files and associated task
def get_GCJ_code(sample_number = 0):
    code = ''
    sample=sample_number
    count=0
    for i in range(len(df['flines'])):
        file_name = df['file'][i]
        file_type = file_name.split('.')[-1]
        if file_type == 'py':
            count+=1
            if sample==count:
                code+=df['flines'][i]
                break
    return code

def centered_sliding_window(token_list, window_diamiter,encode=False,PAD='unk'):
    windows = []
    for i in range(len(token_list)):
        
        #print(token_list)
        #input()
        
        window = []
        
        #if we have to pad the begining
        if i < window_diamiter:
            before_len = window_diamiter-i
            before = [PAD]*before_len+token_list[0:i]
        else:
            before = token_list[i-window_diamiter:i]
        
        #if we have to pad the end
        if i+window_diamiter>=len(token_list):
            after_len = (i+1+window_diamiter)-len(token_list)
            after = token_list[i+1:i+1+window_diamiter]+[PAD]*after_len

        else:
            after = token_list[i+1:i+1+window_diamiter]
        
        #put it togeather
        #print('------')
        #print('before:',before)
        #print('center:',token_list[i])
        #print('after:',after)
        window = before + [token_list[i]] + after
        #for encoding code if we want
        if encode:
            new_window = []
            #print(window)
            #input()
            for i in window:
                encoded = tokenizer.encode(i)
                if len(encoded)>1:       
                    x=encoded[1]
                    if type(x)==list:
                        new_window.append(x[0])
                    else:
                        new_window.append(x)
                elif len(encoded)==1:
                    if type(encoded)==list:
                        new_window.append(encoded[0])
                    else:
                        new_window.append(encoded)
                else:
                    #for some reason it finds the unicode stuff __
                    pass
                    #print(window)
                    #print(i)
                    #print(encoded)
                    #input()
            #print(window)
            #print(len(window))
            #print(len(tokenizer.decode(window)))
            #print(tokenizer.decode(window))
            #print(len(tokenizer.encode(window)))
            #window = tokenizer.encode(tokenizer.decode(window))
            window = new_window
        #print(window)
        #print(len(window))
        #input()

        #save windowz
        windows.append(window)
    
    return windows


###
# adapted from the PyTorch examples. for the full PyTorch LM example, see: 
# https://github.com/pytorch/examples/blob/master/word_language_model/model.py
###

class LSTM_LM(nn.Module):
    """Model feeds pre-trained embeddings through a series of biLSTM
       layers, followed by a linear vocabulary decoder."""
    
    def __init__(self, in_dim, hidden_dim, lstm_layers, word_vectors, 
                 dropout=0.05, bidirectional = True):
        super(LSTM_LM, self).__init__()

        self.vocab_size = word_vectors.shape[0]
        self.hidden_dim = hidden_dim
        self.lstm_layers = lstm_layers

        # blank embed layer starting from GloVe pre-trained vectors
        self._embed = nn.Embedding.from_pretrained(word_vectors, freeze=False)        
        self._drop = nn.Dropout(dropout)

        self._lstm = nn.LSTM(in_dim, hidden_dim, num_layers = lstm_layers, dropout = dropout,
                             bidirectional = bidirectional, batch_first=True)
        self._ReLU = nn.ReLU()
        self._pred = nn.Linear((2 if bidirectional else 1)*hidden_dim, 
                               #self.vocab_size)
                               1) #only 1 or zeros here 

    def forward(self, x):
        e = self._drop(self._embed(x))
        z, h = self._lstm(e)
        z_drop = self._drop(z)
        s = self._pred(self._ReLU(z_drop))
        #s = s.view(-1, self.vocab_size)
        s = s.squeeze()
        return s, h

    def init_hidden(self, batch_size):
        weight = next(self.parameters())
        return weight.new_zeros(self.lstm_layers, batch_size, self.hidden_dim)



In [11]:
########
## use saved model
#######

torch.manual_seed(691)

#vocab size from sentence peice
vocab_size = 10000 #same as sentence peice
vocab_dim = 50  # the size of our pre-trained word vectors

# randomly initialize our word vectors!
vocab_dim = 256
word_vectors = torch.randn(vocab_size, vocab_dim)
word_vectors.shape, word_vectors

#set up model
hidden_dim = 200
lstm_layers = 2
LSTM_LM_net_trained = LSTM_LM(word_vectors.shape[1], hidden_dim,lstm_layers, word_vectors)

#[TODO]: fix so it works
#https://pytorch.org/tutorials/beginner/saving_loading_models.html
#https://stackoverflow.com/questions/61242966/pytorch-attributeerror-function-object-has-no-attribute-copy

name = 'long_py'

class Tokenizer:
    def __init__(self, filepath=name+'_tokenizer.model'):
        self.sp = spm.SentencePieceProcessor(model_file=filepath)

    def encode(self, text, t=int):
        return self.sp.encode(text, out_type=t)

    def decode(self, pieces):
        return self.sp.decode(pieces)

    @staticmethod
    def train(input_file='data/raw_sents.txt', model_prefix='sp_model', vocab_size=30522):
        spm.SentencePieceTrainer.train(input=input_file, model_prefix=model_prefix, vocab_size=vocab_size,
                                        #input_sentence_size=2 ** 16, shuffle_input_sentence=True)
                                        input_sentence_size=number_of_lines, shuffle_input_sentence=True)

#load weights into model
LSTM_LM_net_trained.load_state_dict(torch.load('./data/'+name+'biLSTM_LM.pt'))
LSTM_LM_net_trained.eval()

#instantiate tokenizer model
tokenizer = Tokenizer(name+'_tokenizer.model')


In [29]:
'''
num_samples=1
i = np.random.choice(range(0,segment_length),num_samples)
code_windows = segments[str(i[0])]['x']

code, breaks = get_predicted_break_points(code_windows,LSTM_LM_net_trained)

print(breaks)

#from code segmentation file
comments_added = insert_comments(code,breaks)
comments_added_decoded = tokenizer.decode(comments_added)
comments_added_token_string = ''.join(comments_added_decoded)
comments_added_token_string = comments_added_token_string.replace('SPACE',' ')
comments_added_token_string = comments_added_token_string.replace('NEWLINE','\n')
comments_added_token_string = comments_added_token_string.replace('TAB','\t')
print(comments_added_token_string)
'''

"\nnum_samples=1\ni = np.random.choice(range(0,segment_length),num_samples)\ncode_windows = segments[str(i[0])]['x']\n\ncode, breaks = get_predicted_break_points(code_windows,LSTM_LM_net_trained)\n\nprint(breaks)\n\n#from code segmentation file\ncomments_added = insert_comments(code,breaks)\ncomments_added_decoded = tokenizer.decode(comments_added)\ncomments_added_token_string = ''.join(comments_added_decoded)\ncomments_added_token_string = comments_added_token_string.replace('SPACE',' ')\ncomments_added_token_string = comments_added_token_string.replace('NEWLINE','\n')\ncomments_added_token_string = comments_added_token_string.replace('TAB','\t')\nprint(comments_added_token_string)\n"

In [12]:
#https://stackoverflow.com/questions/54145433/reading-csv-file-line-by-line-and-save-lines-which-are-satisfying-certain-condit
import pandas as pd
df = pd.read_csv("gcj2008.csv")

headers = []

for header in df:
    headers.append(header)
    
print(headers)

['Unnamed: 0', 'year', 'round', 'username', 'task', 'solution', 'file', 'full_path', 'flines']


In [13]:
for i in range(1,100):
    print('-----------------------------------------------------')
    code = get_GCJ_code(sample_number=i)

    new_code = code.replace(' ',' SPACE')
    #newline
    new_code = new_code.replace('\n',' NEWLINE')
    #tab
    new_code = new_code.replace('\t',' TAB')

    tokens = tokenizer.encode(new_code,t=str)


    wd=20 #window diameter
    X_windows = centered_sliding_window(tokens,wd,encode=True)
    code, breaks = get_predicted_break_points(X_windows,LSTM_LM_net_trained)
    
    if len(breaks)>0:
        print(i,len(breaks))
        #'''
        #from code segmentation file
        comments_added = insert_comments(code,breaks)
        comments_added_decoded = tokenizer.decode(comments_added)
        comments_added_token_string = ''.join(comments_added_decoded)
        comments_added_token_string = comments_added_token_string.replace('SPACE',' ')
        comments_added_token_string = comments_added_token_string.replace('NEWLINE','\n')
        comments_added_token_string = comments_added_token_string.replace('TAB','\t')
        print(comments_added_token_string)
        #'''

-----------------------------------------------------
574
[3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 23, 1040, 59, 2094, 59, 704, 59, 294, 3, 680, 4, 3, 4, 28, 3, 526, 86, 7, 43, 9]
[3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 23, 1040, 59, 2094, 59, 704, 59, 294, 3, 680, 4, 3, 4, 28, 3, 526, 86, 7, 43, 9, 3]
[3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 23, 1040, 59, 2094, 59, 704, 59, 294, 3, 680, 4, 3, 4, 28, 3, 526, 86, 7, 43, 9, 3, 86]
[3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 23, 1040, 59, 2094, 59, 704, 59, 294, 3, 680, 4, 3, 4, 28, 3, 526, 86, 7, 43, 9, 3, 86, 12]
[3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 3843, 23, 1040, 59, 2094, 59, 704, 59, 294, 3, 680, 4, 3, 

KeyboardInterrupt: 

# Run on target MSC viaa dataloader for exact accuracy etc

from the training model section where we did the train and validation sets

#### TODO: get test loader modeled on the training section

In [8]:
def run_test():#(source, target):
    
    test_loader = torch.utils.data.DataLoader(LM_test_loader, batch_size=num_test, shuffle=True)
    
    
    #SHOW RESUTLS on all Tokens
    # calculate the fpr and tpr for all thresholds of the classification
    for batch in test_loader:

        #get predicted values
        y_preds, h = LSTM_LM_net_trained(batch['x'])
        y_preds = torch.flatten(torch.sigmoid(y_preds))
        y_preds= y_preds.detach().numpy()

        #get true values
        y = batch['y'].view(-1)
        y = y.detach().numpy()

        #accuracy
        acc = metrics.accuracy_score(y, numpy.rint(y_preds))

        # Compute ROC curve and ROC area for each class
        fpr, tpr, threshold = metrics.roc_curve(y, y_preds)

        roc_auc = metrics.auc(fpr, tpr)


    print('Accuracy:',acc)
    print('Area under curve:',roc_auc)
    plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange',
             lw=lw, label='ROC curve (area = %0.3f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    #plt.xlim([0.0, 1.0])
    #plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic (ROC) on Test Set')
    plt.legend(loc="lower right")
    plt.show()
    
    #instantiate tokenizer model
    
    class Tokenizer:
    
        def __init__(self, filepath=name+'_tokenizer.model'):
            self.sp = spm.SentencePieceProcessor(model_file=filepath)

        def encode(self, text, t=int):
            return self.sp.encode(text, out_type=t)

        def decode(self, pieces):
            return self.sp.decode(pieces)

        @staticmethod
        def train(input_file='data/raw_sents.txt', model_prefix='sp_model', vocab_size=30522):
            spm.SentencePieceTrainer.train(input=input_file, model_prefix=model_prefix, vocab_size=vocab_size,
                                           #input_sentence_size=2 ** 16, shuffle_input_sentence=True)
                                           input_sentence_size=number_of_lines, shuffle_input_sentence=True)

    
    tokenizer = Tokenizer(name+'_tokenizer.model')
    
    
    #make a _NEW_LINE_ ROC curve since thats what we care about
    test_loader = torch.utils.data.DataLoader(LM_test_loader, batch_size=num_test, shuffle=True)

    # calculate the fpr and tpr for all thresholds of the classification
    for batch in test_loader:

        #get predicted values
        y_preds, h = LSTM_LM_net_trained(batch['x'])
        y_preds = torch.flatten(torch.sigmoid(y_preds))
        y_preds= y_preds.detach().numpy()

        #get true values
        y = batch['y'].view(-1)
        y = y.detach().numpy()

        #filter for only new line places (get info from batch['x'])
        new_y=[]
        new_y_preds=[]
        tokens = torch.flatten(batch['x']).detach().numpy()
        for i in range(len(tokens)):
            tok_literal=tokenizer.decode(int(tokens[i]))
            if tok_literal[-7:]=='NEWLINE':
                #only rate actual break places
                #if y[i]==1:
                #    new_y.append(y[i])
                #    new_y_preds.append(y_preds[i])
                new_y.append(y[i])
                new_y_preds.append(y_preds[i])
        #accuracy
        #https://numpy.org/doc/stable/reference/generated/numpy.rint.html
        acc = metrics.accuracy_score(new_y, numpy.rint(new_y_preds))

        # Compute ROC curve and ROC area for each class
        fpr, tpr, threshold = metrics.roc_curve(new_y, new_y_preds)

        roc_auc = metrics.auc(fpr, tpr)

    print('Accuracy:',acc)
    print('Area under curve:',roc_auc)
    plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange',
             lw=lw, label='ROC curve (area = %0.3f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    #plt.xlim([0.0, 1.0])
    #plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic (ROC) on Test Set')
    plt.legend(loc="lower right")
    plt.show()
    
run_test()

NameError: name 'LM_test_loader' is not defined